In [106]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OneHotEncoder  ##. better to use dummy from pandas 
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
import statsmodels.api as sm

## Instructions

**Instructions**

- fit the models LinearRegressor,Lasso,Ridge and (optional KNeighborsRegressor ) and compare the model performances. ✅
- (Optional) Define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code. ✅
- Use feature selection techniques(P-Value, RFE) to select subset of features to train the model with(if necessary). ✅
- (optional) refit the models with the selected features. ✅


## Data Preprocessing & Transformation

### Extract / load data

In [59]:
def load_original_data():
    df = pd.read_csv(r"C:\Users\p.kollhof\Documents\IRONHACK_GitHub\IH_RH_DA_FT_AUG_2022\Class_Materials\Case_Studies\Customer_Analysis_Case_Study\Data\Data_Marketing_Customer_Analysis_Round3.csv")        
    return df

df_original = load_original_data()

### Preprocessing / transforming data

#### `Split` numerical and categorical `data`

In [42]:
def get_numerical(df):
    df_numerical = df.select_dtypes(exclude=['object'])
    # Drop number of open complaints as this is technically categorical data
    return df_numerical

def get_categorical(df):
    df_categorical = df.select_dtypes(include=['object'])
    return df_categorical

df_numerical = get_numerical(df_original)
df_categorical = get_categorical(df_original)

#### `Numerical` data `processing`

In [43]:
# Drop "number_of_open_complaints", because it's technically categorical data
df_numerical.drop("number_of_open_complaints", axis=1, inplace=True)

"""
- not checking for outliers in customer_lifetime_value, because also extreme cases are reasonable
- 'number_of_policies'
"""
df_num_cols_selected = ['income', 'monthly_premium_auto', 'months_since_last_claim', 
                        'months_since_policy_inception']

# Get rid of outliers if there are any
def remove_outliers(df,columns,n_std):
    """
    function for removing outliers from given df;
    select in which column outliers are to be removed;
    specify "outlier condition" by supplying number of standard deviations;
    """
    for col in columns:
        mean = df[col].mean()
        std = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*std))]
        #df.reset_index(drop=True, inplace=True)
        
    return df

df_numerical_nofliers = remove_outliers(df_numerical, df_num_cols_selected, 3)

#### `Categorical` data `processing`

In [44]:
def transform_categorical(df):
    # Drop effective to date column
    df_categorical.drop("effective_to_date", axis=1, inplace=True) 
    
    df = load_original_data()
    # Insert number of open complaints as this is technically categorical data
    df_categorical.insert(len(df_categorical.columns)-1, "number_of_open_complaints", df["number_of_open_complaints"])
    
    # Convert policy to numbers for ordinal
    policy_dict = {"personal l1": 1, "personal l2": 2, "personal l3": 3,
                    'corporate l1': 4, 'corporate l2': 5, 'corporate l3': 6,
                    'special l1': 7, 'special l2': 8, 'special l3': 9}
    #policies = [policy_dict[policy] for policy in df_categorical["policy"]]
    ## Group policies
    policies = ["personal" if pol in ("personal l1", "personal l2", "personal l3")
               else "corporate" if pol in ("corporate l1", "corporate l2", "corporate l3")
               else "special" for pol in df_categorical["policy"]]
    df_categorical["policy"] = policies
    
    # Convert education  to numbers for ordinal
    education_dict = {'high school or below': 0,
                      'bachelor': 1,
                      'master': 2,
                      'college': 2,
                      'doctor': 3}
    educations = [education_dict[edu] for edu in df_categorical["education"]]
    df_categorical["education"] = educations
    
    # Convert coverage to numbers for ordinal
    df_categorical["coverage"] = [1 if x == "basic"
                                 else 2 if x == "extended"
                                 else 3
                                 for x in df_categorical["coverage"]]
    
    # Convert months to numbers for ordinal
    #months_dict = {"jan": 0,
              #"feb": 1}
    #df_categorical["month"] = [months_dict[mon] for mon in df_categorical["month"]]
    
    # Group luxury cars together
    df_categorical["vehicle_class"] = ["luxury" if car in ("luxury car", "luxury suv")
                                      else car
                                      for car in df_categorical["vehicle_class"]]
    
    return df_categorical

df_categorical = transform_categorical(df_categorical)

#### `Split categorical` into nominal and ordinal

In [45]:
df_categorical_ordinal = df_categorical[["education", "policy", "number_of_open_complaints", "coverage"]]
df_categorical_nominal = df_categorical.drop(columns=["education", "policy", "number_of_open_complaints", "coverage"])

### One Hot/Label Encoding of the categorical variables

In [48]:
df_categorical_hotcoded = pd.get_dummies(df_categorical, drop_first=True) # 43 columns
# df_categorical_hotcoded_ = pd.get_dummies(df_categorical, drop_first=False) # 58 columns

df_categorical_ordinal_hotcoded = pd.get_dummies(df_categorical_ordinal, drop_first=True)
df_categorical_nominal_hotcoded = pd.get_dummies(df_categorical_nominal, drop_first=True)

df_categorical_combined = pd.concat((df_categorical_ordinal_hotcoded,df_categorical_nominal_hotcoded), axis=1)

### Combine numerical and encoded categoricals

In [58]:
df_numerical_categorical = pd.concat((df_categorical_hotcoded, df_numerical), axis=1)
df_numerical_nofliers_categorical = pd.merge(df_categorical_hotcoded, df_numerical_nofliers, left_index=True, right_index=True)

df = df_numerical_categorical
df

,coverage,education,number_of_open_complaints,region_east,region_north west,region_west region,response_yes,month_jan,employment_status_employed,employment_status_medical leave,...,vehicle_class_two-door car,vehicle_size_medsize,vehicle_size_small,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_policies,total_claim_amount
0,1,2,0,0,0,0,0,0,1,0,...,0,1,0,4809,48029,61,7,52,9,292
1,1,2,0,0,0,1,0,1,0,0,...,0,1,0,2228,92260,64,3,26,1,744
2,1,1,0,1,0,0,0,0,1,0,...,0,1,0,14947,22139,100,34,31,2,480
3,2,2,0,0,1,0,1,1,1,0,...,0,1,0,22332,49078,97,10,3,2,484
4,3,1,0,0,1,0,0,1,0,1,...,0,1,0,9025,23675,117,33,31,7,707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,3,1,0,0,0,0,0,1,0,0,...,0,1,0,15563,61541,253,12,40,7,1214
10685,1,2,0,0,1,0,0,1,1,0,...,0,1,0,5259,61146,65,7,68,6,273
10686,2,1,0,0,0,0,0,0,1,0,...,0,1,0,23893,39837,201,11,63,2,381
10687,3,2,4,0,0,1,0,0,1,0,...,0,1,0,11971,64195,158,0,27,6,618


## Linear Regression Modeling

In [102]:
def X_y_split(df, column):
    
    X = df.drop('total_claim_amount', axis=1)
    y = df["total_claim_amount"]
    
    return X, y

In [126]:
def test_train_split_(df, test_size = 0.3, randomstate = 42):

    X, y = X_y_split(df, 'total_claim_amount')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=42)
    
    return X, y, X_train, X_test, y_train, y_test

### Linear Regressor

In [51]:
model=LinearRegression()
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {round(model.score(X_train, y_train),4)}, Test -> {round(model.score(X_test, y_test),4)}")

LinearRegression: Train -> 0.7652, Test -> 0.7826


### Lasso

In [52]:
model=Lasso(alpha=0.05)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {round(model.score(X_train, y_train),4)}, Test -> {round(model.score(X_test, y_test),4)}")

Lasso: Train -> 0.7652, Test -> 0.7827


### Ridge

In [57]:
model=Ridge(alpha=10)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {round(model.score(X_train, y_train),4)}, Test -> {round(model.score(X_test, y_test),4)}")

Ridge: Train -> 0.7651, Test -> 0.7827


### Run all models in function

In [149]:
X, y, X_train, X_test, y_train, y_test = test_train_split_(df)

def run_linear_lasso_ridge_regressions(X_train, X_test, y_train, y_test, n_features, rfe = False):

    model_list = (LinearRegression(), Lasso(alpha=0.05), Ridge(alpha=10))

    for model in model_list:
        if rfe:
            selector = RFE(model, n_features_to_select = n_features, step = 1, verbose = 1) # Step is how many features to add or drop everytime
            selector.fit(X_train, y_train)

            kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
            kept_features = list(X_train.iloc[:,kept_features].columns)

            X_train = selector.transform(X_train)
            X_test  = selector.transform(X_test)

            X_train = pd.DataFrame(X_train, columns=kept_features)
            X_test  = pd.DataFrame(X_test, columns=kept_features)

            #print("Final selected features: ")
        model.fit(X_train, y_train)
        print()
        print(f"{model.__class__.__name__}: Train -> {round(model.score(X_train, y_train),4)}, Test -> {round(model.score(X_test, y_test),4)}")
        

## Feature Selection Techniques (P-Value, RFE)

### Recursive Feature Elimination

In [150]:
run_linear_lasso_ridge_regressions(X_train, X_test, y_train, y_test, 16, True)

Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.

LinearRegression: Train -> 0.7154, Test -> 0.7458

Lasso: Train -> 0.7154, Test -> 0.7457

Ridge: Train -> 0.7151, Test -> 0.7444


### P-Value

In [168]:
def drop_features_by_p_value(multiround = False):
    
    # Load X, y data
    X, y = X_y_split(df, 'total_claim_amount')
    
    # Add constant (intercept) to X
    X_added_constant = sm.add_constant(X)
    
    # Calculate stats with OLS model
    model = sm.OLS(y, X_added_constant).fit()
    display(model.summary())
    
    # Find features with P > 0.05
    significant_features = model.params[list(np.where(model.pvalues < 0.05)[0])].iloc[0:].index.tolist()
    
    # Only keep significant features (P <= 0.05)
    X_added_constant_only_sig_features = X_added_constant[significant_features]
    display(X_added_constant_only_sig_features.shape)
    
    #if multiround:
    # Calculate stats with OLS model
    model_2 = sm.OLS(y, X_added_constant_only_sig_features).fit()
    display(model_2.summary())

    # Find features with P > 0.05
    significant_features = model_2.params[list(np.where(model_2.pvalues < 0.05)[0])].iloc[0:].index.tolist()

    if len(significant_features):
        # Only keep significant features (P <= 0.05)
        X_added_constant_only_sig_features = X_added_constant[significant_features]
    display(X_added_constant_only_sig_features.shape)
    
    return X_added_constant_only_sig_features

In [169]:
X_train, X_test, y_train, y_test = train_test_split(drop_features_by_p_value(), y, test_size=0.2,random_state=42)

run_linear_lass_ridge_regressions(X_train, X_test, y_train, y_test)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     967.1
Date:                Mon, 19 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:57:23   Log-Likelihood:                -67996.
No. Observations:               10689   AIC:                         1.361e+05
Df Residuals:                   10651   BIC:                         1.363e+05
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                            -338.3764     12.270    -27.578      0.000    -362.428    -314.325
coverage                           -8.7861      3.354     -2.619      0.009     -15.361      -2.211
education                          -9.0672      1.559     -5.818      0.000     -12.122      -6.012
number_of_open_complaints          -1.7048      1.497     -1.139      0.255      -4.640       1.230
region_east                         5.9820      5.267      1.136      0.256      -4.342      16.306
region_north west                  -1.8772      3.603     -0.521      0.602      -8.940       5.186
region_west region                  3.7740      3.435      1.099      0.272      -2.959      10.507
response_yes                      -24.3915      4.393     -5.552      0.000     -33.003     -15.780
month_jan                          -0.7160      2.728     -0.262      0.793      -6.064       4.632
employment_status_employed        -11.6942      7.044     -1.660      0.097     -25.502       2.114
employment_status_medical leave    11.6318      8.991      1.294      0.196      -5.992      29.256
employment_status_retired         -10.3200     10.543     -0.979      0.328     -30.987      10.347
employment_status_unemployed       77.5485      7.444     10.418      0.000      62.957      92.140
gender_m                           15.6954      2.745      5.717      0.000      10.314      21.077
location_code_suburban            380.2467      3.930     96.765      0.000     372.544     387.949
location_code_urban               222.1250      4.485     49.527      0.000     213.334     230.916
marital_status_married             -4.9589      4.041     -1.227      0.220     -12.880       2.963
marital_status_single              66.4875      4.639     14.334      0.000      57.395      75.580
policy_type_personal auto           1.0722      1.662      0.645      0.519      -2.185       4.329
policy_type_special auto            6.4458      3.662      1.760      0.078      -0.732      13.623
policy_personal                     1.0722      1.662      0.645      0.519      -2.185       4.329
policy_special                      6.4458      3.662      1.760      0.078      -0.732      13.623
renew_offer_type_offer2            15.3940      3.412      4.512      0.000       8.706      22.082
renew_offer_type_offer3             8.5383      4.094      2.086      0.037       0.514      16.563
renew_offer_type_offer4             6.9135      4.799      1.441      0.150      -2.493      16.320
sales_channel_branch                1.1270      3.417      0.330      0.742      -5.571       7.825
sales_channel_call center           0.1763      3.830      0.046      0.963      -7.332       7.684
sales_channel_web                   2.5780      4.219      0.611      0.541      -5

(10689, 16)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.770
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     2382.
Date:                Mon, 19 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:57:23   Log-Likelihood:                -68011.
No. Observations:               10689   AIC:                         1.361e+05
Df Residuals:                   10673   BIC:                         1.362e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                         -347.3928      6.974    -49.815      0.000    -361.062    -333.723
coverage                        -9.2972      2.408     -3.860      0.000     -14.018      -4.576
education                       -9.0177      1.555     -5.798      0.000     -12.067      -5.969
response_yes                   -25.2824      4.199     -6.021      0.000     -33.514     -17.051
employment_status_unemployed    86.3452      3.649     23.660      0.000      79.192      93.499
gender_m                        15.6523      2.732      5.729      0.000      10.297      21.007
location_code_suburban         382.7337      3.837     99.751      0.000     375.213     390.255
location_code_urban            222.0741      4.481     49.556      0.000     213.290     230.858
marital_status_single           69.7911      3.365     20.741      0.000      63.195      76.387
renew_offer_type_offer2         13.2047      3.129      4.220      0.000       7.071      19.339
renew_offer_type_offer3          6.4922      3.905      1.663      0.096      -1.162      14.146
vehicle_class_sports car       -24.2031      6.369     -3.800      0.000     -36.688     -11.718
vehicle_class_suv              -17.4661      3.861     -4.524      0.000     -25.035      -9.898
income                          -0.0003   5.75e-05     -4.487      0.000      -0.000      -0.000
monthly_premium_auto             5.2331      0.051    102.782      0.000       5.133       5.333
months_since_last_claim          0.3520      0.135      2.610      0.009       0.088       0.616
==============================================================================
Omnibus:                     4715.505   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            58690.492
Skew:                           1.781   Prob(JB):                         0.00
Kurtosis:                      13.913   Cond. No.                     3.21e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.21e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

(10689, 15)


LinearRegression: Train -> 0.7691, Test -> 0.773

Lasso: Train -> 0.7691, Test -> 0.7729

Ridge: Train -> 0.7691, Test -> 0.7728
